In [ ]:
# Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen
from urllib.request import Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('all')

# Parameters 
n = 10 #the # of article headlines displayed per ticker
tickers = ['V', 'MA', 'PYPL','AXP','COF','BAC','JPM','WFC','C','BRK-B','DFS','SYF','WU','AIG','HIG','PFG','TFC','USB','FRC','SIVB','FITB','MTB','BBVA','BCS',
           'BMO','BNS','CM','CS','EWBC','HSBC','CFG','RF','HBAN','PBCT','CMA','ZION','RY','SAN','SMFG','TD']

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

In [ ]:
# Get Data
finviz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finviz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass



Recent News Headlines for V: 
Why is OnlyFans letting payment companies lure it away from porn? ( Aug-21-21 07:30AM )
Is a Stock Market Crash Coming? Here's What the Data Suggests ( 05:51AM )
Follow in Buffett's Footsteps by Retaining 3 Payment Stocks ( Aug-18-21 09:27AM )
Bitcoin down even as crypto industry undergoes a 'revolution' ( 07:22AM )
Top Stock Reports for Visa, Morgan Stanley & Royal Dutch Shell ( Aug-13-21 01:12PM )
Mastercard (MA) Teams Up to Aid Mpamba's Mobile Money Clients ( Aug-12-21 09:42AM )
3 Dividend-Paying Tech Stocks to Buy in August ( 06:40AM )
Caterpillar, Home Depot, and Walgreens Lift Dow Jones to Record Even as Visa Stock Falls ( Aug-11-21 04:36PM )
Visa U.S. Spending Momentum Index Picks up in July ( 10:00AM )
Visa Inc.'s (NYSE:V) Recent Stock Performance Looks Decent- Can Strong Fundamentals Be the Reason? ( 08:38AM )


Recent News Headlines for MA: 
Why is OnlyFans letting payment companies lure it away from porn? ( Aug-21-21 07:30AM )
As Covid Lingers

In [ ]:
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        text = x.a.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text])

In [ ]:
analyzer = SentimentIntensityAnalyzer()

columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns=columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix='_right')
news['Date'] = pd.to_datetime(news.Date).dt.date

In [ ]:
news

In [ ]:
type = []
for new in news['compound'] : 
  if new == 0 :
    type.append('neutral')
  elif new > 0 :
    type.append ('positive')
  else :
    type.append('negative')

print(type)
news['sentiment'] = type

['positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'negative', 'neutral', 'negative', 'positive', 'positive', 'neutral', 'neutral', 'neutral', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'neutral', 'positive', 'neutral', 'positive', 'positive', 'neutral', 'neutral', 'positive', 'positive', 'positive', 'negative', 'neutral', 'neutral', 'neutral', 'positive', 'positive', 'negative', 'neutral', 'neutral', 'positive', 'positive', 'negative', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'neutral', 'neutral', 'positive', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'positive', 'neutral', 'positive', 'neutral', 'positive', 'positive', 'positive', 'negative', 'pos

In [ ]:
news

,Ticker,Date,Time,Headline,neg,neu,pos,compound,sentiment
0,V,2021-07-22,03:02PM,AXP Option Traders Charged Up for Positive Ear...,0.158,0.526,0.316,0.4215,positive
1,V,2021-07-22,12:46PM,10 REIT Dividend Stocks with Over 5% Yields,0.000,1.000,0.000,0.0000,neutral
2,V,2021-07-22,07:22AM,Visa snaps up London fintech Currencycloud for...,0.000,1.000,0.000,0.0000,neutral
3,V,2021-07-22,05:00AM,Visa to Acquire Currencycloud,0.000,1.000,0.000,0.0000,neutral
4,V,2021-07-22,04:06AM,"Texas Instruments, DR Horton Fall Premarket; A...",0.000,1.000,0.000,0.0000,neutral
...,...,...,...,...,...,...,...,...,...
3995,TD,2021-03-01,08:50AM,Toronto-Dominion (TD) Stock Down 5.1% Despite ...,0.000,1.000,0.000,0.0000,neutral
3996,TD,2021-03-01,07:30AM,What Is A Dividend? Plus The 5 Best Dividend S...,0.000,0.656,0.344,0.6369,positive
3997,TD,2021-02-26,05:30PM,TD Bank Group (TD) Q4 2020 Earnings Call Trans...,0.000,1.000,0.000,0.0000,neutral
3998,TD,2021-02-25,11:30AM,The Toronto-Dominion Bank to Host Earnings Call,0.000,1.000,0.000,0.0000,neutral


In [ ]:
news.to_excel("finfiz.xlsx")